<a href="https://colab.research.google.com/github/DeependraChaddha/Heat_Equation_Solver/blob/main/Heat_Equation_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

Using Gradient Descent to solve heat equation.
The difference between LHS and RHS of Heat Equation is used as Loss function.
Will experiment with optimizers to see which works best

#Make imports

In [13]:
import torch
from torch import nn
import matplotlib
import numpy as np
import random
from torch import Tensor

#Device-Agnostic Code

In [14]:
device='cuda' if torch.cuda.is_available() else 'cpu'

Set random seeds

In [15]:
def set_seed(seed=42):
    random.seed(seed)  # Python random module
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch CUDA (Single GPU)
    torch.cuda.manual_seed_all(seed)  # PyTorch CUDA (All GPUs)

    # Ensures deterministic behavior in GPU operations
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # Can slow down training but ensures reproducibility

# Call this at the beginning of your script
set_seed(42)

#Specifying equation and coordinate system

Will figure out how to solve for first example, will then functionalize and implement on example 2 and 3. Example 4 is time dependent so may have to use different approach for that

##Example 1
∂²T/∂x² + ∂²T/∂y² = 8π²sin(2πx)sin(2πy)

T(0,y) = 0
T(1,y) = 0
T(x,0) = 0
T(x,1) = 0


Take x=0,x=1, y=0,y=1 as edges because of the initial conditions

In [16]:
#Specify number of points in grid in each direction
#todo assert both nx,ny to be strictly greater than 1 when generalizing
nx=10
ny=10

#calculate spacing between 2 points in the grid
lx=1/(nx-1)
ly=1/(ny-1)

Make grid

In [17]:
#Make linear space of x and y and reshape to the shape(nx,1) and (ny,1) respectively (Reshaping is done so that in case of matrix multiplication or broadcasting no error is encountered)
x=torch.linspace(0,1,nx).reshape(-1,1).requires_grad_(True)
y=torch.linspace(0,1,ny).reshape(-1,1).requires_grad_(True)

#Make meshgrid
X,Y=torch.meshgrid(x.squeeze(),y.squeeze(),indexing='ij')
X.reshape(-1,1).requires_grad_(True)
Y.reshape(-1,1).requires_grad_(True)
print(X.shape)
print(Y.shape)

torch.Size([10, 10])
torch.Size([10, 10])


In [18]:
#To get (i,j) point
i=3
j=4
print(f'(i,j)th point is {X[i,j]},{Y[i,j]}')

(i,j)th point is 0.3333333432674408,0.4444444477558136


###Make PINN to find solution of Heat Equation







In [19]:
class SolutionFunction(nn.Module):
  def __init__(self):
    set_seed()
    super().__init__()
    self.layers=nn.Sequential(
        nn.Linear(2,10),
        nn.Tanh(),
        nn.Linear(10,10),
        nn.Tanh(),
        nn.Linear(10,10),
        nn.Tanh(),
        nn.Linear(10,1)
    )

  def forward(self,X,Y):
      #reshape before concatenation to get correct dimension
      X=X.reshape(-1,1)
      Y=Y.reshape(-1,1)
      return self.layers(torch.cat((X,Y),dim=1)).reshape(10,10)

#check if it works properly
demo_model=SolutionFunction()
demo_model(X,Y)

tensor([[0.3230, 0.3151, 0.3071, 0.2989, 0.2908, 0.2827, 0.2748, 0.2672, 0.2598,
         0.2527],
        [0.3271, 0.3189, 0.3105, 0.3021, 0.2937, 0.2854, 0.2773, 0.2695, 0.2621,
         0.2550],
        [0.3313, 0.3227, 0.3141, 0.3054, 0.2968, 0.2883, 0.2801, 0.2722, 0.2647,
         0.2576],
        [0.3355, 0.3267, 0.3178, 0.3089, 0.3001, 0.2915, 0.2832, 0.2752, 0.2676,
         0.2605],
        [0.3397, 0.3307, 0.3217, 0.3126, 0.3037, 0.2950, 0.2866, 0.2785, 0.2709,
         0.2638],
        [0.3438, 0.3347, 0.3256, 0.3164, 0.3074, 0.2987, 0.2902, 0.2821, 0.2745,
         0.2673],
        [0.3479, 0.3387, 0.3295, 0.3203, 0.3113, 0.3025, 0.2941, 0.2860, 0.2783,
         0.2712],
        [0.3517, 0.3426, 0.3334, 0.3243, 0.3154, 0.3066, 0.2982, 0.2901, 0.2825,
         0.2753],
        [0.3553, 0.3464, 0.3373, 0.3284, 0.3195, 0.3108, 0.3025, 0.2945, 0.2869,
         0.2797],
        [0.3588, 0.3500, 0.3412, 0.3324, 0.3237, 0.3152, 0.3070, 0.2991, 0.2915,
         0.2844]], grad_fn=<

In [25]:
demo_model(X,Y).shape[-1]

10

Constructing loss functions

In [42]:
def pde_loss(model:nn.Module,
             x:torch.Tensor,
             y:torch.Tensor):
  '''
  calculates loss according to the heat equation
  Inputs:
  Model:the neural network being trained
  x,y: x,y direction coordinates of meshgrid

  Output: Value of loss according to heat equation
  '''

  x.requires_grad_(True)
  y.requires_grad_(True)

  #calculate meshgrid
  f=model(X,Y)

  #Calculate gradients
  f_x=torch.autograd.grad(f,x,torch.ones_like(f),create_graph=True)[0]
  f_y=torch.autograd.grad(f,y,torch.ones_like(f),create_graph=True)[0]

  #Calculate second order gradients
  f_xx=torch.autograd.grad(f_x,x,torch.ones_like(f_x),create_graph=True)[0]
  f_yy=torch.autograd.grad(f_y,y,torch.ones_like(f_y),create_graph=True)[0]

  #Calculate target value of function
  target=8*np.pi**2*torch.sin(2*np.pi*x)*torch.sin(2*np.pi*y)

  #Calculate loss
  loss=torch.mean((f_xx+f_yy-target)**2)

  return loss




def boundary_loss(model:nn.Module,
                  x:torch.Tensor,
                  y:torch.Tensor):
  ''' Input: the gird coordinates and model
      Output:Boundary Loss Value
      loss is defined for a specific boundary condition will have to change function depending on the problem, it is NOT a generally usable function
  '''
  x.requires_grad_(True)
  y.requires_grad_(True)

  #calculate meshgrid
  f=model(X,Y)

  #get loss
  loss=(f[0,:]-torch.zeros(f.shape[-1]))**2+ (f[-1,:]-torch.zeros(f.shape[-1]))**2+ (f[:,0]-torch.zeros(f.shape[-1]))**2+ (f[:,-1]-torch.zeros(f.shape[-1]))**2

  # return loss
  return torch.mean(loss)






def total_loss(model:nn.Module,
               x:torch.Tensor,
               y:torch.Tensor,
               boundary_loss_weight:float=100.0):
  '''Combining both loss functions using a specific weight to measure relative importance of the 2 losses'''
  return pde_loss(model,x,y)+boundary_loss_weight*boundary_loss(model,x,y)

test loss function

In [43]:
loss=total_loss(demo_model,X,Y)
print(loss)

tensor(1299.7664, grad_fn=<AddBackward0>)


Set Optimizer and Train


In [ ]:
#Try plotting after every few steps to visualize the training process
#Initialize Model
model=SolutionFunction()

#Set SGD Optimizer
sgd_optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

#Set ADAM Optimizer
adam_optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

Make Training Loop

##Example 2
Heat Equation: ∂²T/∂x² + ∂²T/∂y² = 0

Initial Conditions:
T(0,y) = 0
T(1,y) = 0
T(x,0) = 0
T(x,1) = sin(πx)


##Example 3
Heat Equation: ∂²T/∂x² + ∂²T/∂y² = 0

Initial Conditions:
T(0,y) = 0
T(1,y) = y(1-y)
T(x,0) = 0
T(x,1) = 0


##Example 4
Heat Equation: ∂T/∂t = α(∂²T/∂x² + ∂²T/∂y²)

Initial Conditions:
T(x,y,0) = sin(πx)sin(πy)
T(0,y,t) = 0
T(1,y,t) = 0
T(x,0,t) = 0
T(x,1,t) = 0
α = 0.01
